In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb
import numpy
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [87]:
from pathlib import Path

dossier = Path(__file__).resolve().parent.parent / 'data' / 'DIC'

fichiers = [f.name for f in dossier.iterdir() if f.is_file()]

print(fichiers)

NameError: name '__file__' is not defined

In [4]:
doc_path = "../data/DIC/Allianz.pdf"

In [89]:
original_document = PyPDFLoader(doc_path).load()

In [90]:
content = original_document[0].page_content
print(content[0:500])

 I n f o r m a t i o n s   c l Ø s   p o u r   l  i n v e s t i s s e u r
 C e   d o c u m e n t   f o u r n i t   d e s   i n f o r m a t i o n s   e s s e n t i e l l e s   a u x   i n v e s t i s s e u r s   d e   c e t   O P C V M .   I l   n e   s  a g i t   p a s   d  u n   d o c u m e n t   p r o m o t i o n n e l .   L e s
 i n f o r m a t i o n s   q u  i l   c o n t i e n t   v o u s   s o n t   f o u r n i e s   c o n f o r m Ø m e n t   à   u n e   o b l i g a t i o n   l Ø g a l


In [93]:
original_document[1].metadata

{'producer': 'Actuate PDF Writer (Low Resolution) 2.1',
 'creator': 'Actuate e.Reports',
 'creationdate': '2022-07-29T08:11:45+01:00',
 'title': '',
 'subject': '',
 'author': 'IDS GmbH - Analysis and Reporting Services',
 'keywords': 'FR0010032326 (22.08.2022)',
 'source': '../data/DIC/Allianz.pdf',
 'total_pages': 2,
 'page': 1,
 'page_label': '2'}

In [88]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(original_document)

In [8]:
len(chunks)

22

In [9]:
print(chunks[0].page_content)

 I n f o r m a t i o n s   c l Ø s   p o u r   l  i n v e s t i s s e u r
 C e   d o c u m e n t   f o u r n i t   d e s   i n f o r m a t i o n s   e s s e n t i e l l e s   a u x   i n v e s t i s s e u r s   d e   c e t   O P C V M .   I l   n e   s  a g i t   p a s   d  u n   d o c u m e n t   p r o m o t i o n n e l .   L e s
 i n f o r m a t i o n s   q u  i l   c o n t i e n t   v o u s   s o n t   f o u r n i e s   c o n f o r m Ø m e n t   à   u n e   o b l i g a t i o n   l Ø g a l e ,   a f i n   d e   v o u s   a i d e r   à   c o m p r e n d r e   e n   q u o i
 c o n s i s t e   u n   i n v e s t i s s e m e n t   d a n s   c e t   O P C V M   e t   q u e l s   r i s q u e s   y   s o n t   a s s o c i Ø s .   I l   v o u s   e s t   c o n s e i l l Ø   d e   l e   l i r e   p o u r   d Ø c i d e r   e n
 c o n n a i s s a n c e   d e   c a u s e   d  i n v e s t i r   o u   n o n .
 A L L I A N Z   E U R O   H I G H   Y I E L D
 P a r t   R C   ( E U R )


In [10]:
client = chromadb.HttpClient(host='localhost', port=8000)
collection = client.get_or_create_collection(name="financial_reports")

In [49]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

for i, chunk in enumerate(chunks):
    embedding = embeddings_model.embed_query(chunk.page_content)
    collection.upsert(
        embeddings=[embedding],
        documents=[chunk.page_content],
        ids=[f"chunk_{i}"],
        metadatas=[{"filename": doc_path}]
    )

print("Embeddings stockés avec succès dans ChromaDB.")

Embeddings stockés avec succès dans ChromaDB.


In [96]:
import logging

logging.info("TESt")

In [50]:
document_id = "chunk_1"  

result = collection.get(
    ids=[document_id],
    include=["metadatas", "documents", "embeddings"]
)

if result['documents']:
    print(f"Document avec l'ID {document_id} :")
    print(result['documents'][0])
    if result['metadatas'][0]:
        print("Métadonnées :")
        print(result['metadatas'][0])
else:
    print(f"Aucun document trouvé avec l'ID {document_id}")

Document avec l'ID chunk_1 :
 i n f o r m a t i o n s   q u  i l   c o n t i e n t   v o u s   s o n t   f o u r n i e s   c o n f o r m Ø m e n t   à   u n e   o b l i g a t i o n   l Ø g a l e ,   a f i n   d e   v o u s   a i d e r   à   c o m p r e n d r e   e n   q u o i
 c o n s i s t e   u n   i n v e s t i s s e m e n t   d a n s   c e t   O P C V M   e t   q u e l s   r i s q u e s   y   s o n t   a s s o c i Ø s .   I l   v o u s   e s t   c o n s e i l l Ø   d e   l e   l i r e   p o u r   d Ø c i d e r   e n
Métadonnées :
{'filename': '../data/DIC/Allianz.pdf'}


In [51]:
results = collection.query(
    query_texts=["What is name of the company ?"],
    n_results=2
)

In [52]:
print(results["documents"][0][0])

 M a r c h Ø s   F i n a n c i e r s .   A l l i a n z   G l o b a l   I n v e s t o r s   G m b H   e s t   a g r Ø Ø e
 e n   A l l e m a g n e   e t   r Ø g l e m e n t Ø e   p a r   l a   B u n d e s a n s t a l t   f ü r
 F i n a n z d i e n s t l e i s t u n g s a u f s i c h t   ( B a F i n ) .
 L e s   i n f o r m a t i o n s   c l Ø s   p o u r   l  i n v e s t i s s e u r   f o u r n i e s   i c i   s o n t   e x a c t e s
 e t   à   j o u r   a u   2 2 . 0 8 . 2 0 2 2 .


In [30]:
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")
huggin_face_key= getenv("HUGGINGFACEHUB_API_TOKEN")

In [53]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import HuggingFacePipeline



# Création d'un objet Chroma pour la récupération
retriever = Chroma(client=client, collection_name="financial_reports", embedding_function=embeddings_model)

# Création d'un modèle de génération
model_name = "google/flan-t5-small"
llm = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task="text2text-generation",
    model_kwargs={"max_length": 512}
)

from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo")


prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)
# Utilisation de la RAG chain pour répondre à une question
qa_chain.invoke("Que signifie OPCVM ?")

Device set to use cpu
/Users/aurelien/Projects/blentai/chatbot-finance/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"OPCVM signifie Organisme de Placement Collectif en Valeurs Mobilières. Il s'agit d'un fonds d'investissement collectif géré par une société de gestion. Les OPCVM permettent de diversifier les investissements des épargnants."

In [77]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import HuggingFacePipeline



# Création d'un objet Chroma pour la récupération
retriever = Chroma(client=client, collection_name="financial_reports", embedding_function=embeddings_model)

# Création d'un modèle de génération
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

# Définissez le nom du modèle
# Nom du modèle
model_name = "dbddv01/gpt2-french-small"

# Chargement du tokenizer et du modèle
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Création du pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100, 
    temperature=0.7,      # Contrôle la créativité
    top_p=0.9,            # Nucleus sampling
    top_k=50              # Filtrage top-k
)

llm = HuggingFacePipeline(pipeline=generator)

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever.as_retriever(search_kwargs={"k": 10}) | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)
# Utilisation de la RAG chain pour répondre à une question
qa_chain.invoke("Que signifie OPCVM ?")

Device set to use mps:0
/Users/aurelien/Projects/blentai/chatbot-finance/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

# Lien vers le modèle GGUF (préalablement téléchargé)
model_path = "../data/models/mistral-7b-instruct-v0.2.Q4_0.gguf"

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.7,
    top_p=0.9,
    n_ctx=2048,
    max_tokens=256,
    n_threads=4,
    verbose=True,
)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Contexte : {context}\n\nQuestion : {question}\nRéponse :"
)

def format_docs(docs, max_tokens=1500):
    content = "\n\n".join(doc.page_content for doc in docs)
    tokens = tokenizer.encode(content)
    return tokenizer.decode(tokens[:max_tokens])

qa_chain = (
    {
        "context": retriever.as_retriever(search_kwargs={"k": 5}) | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

response = qa_chain.invoke("Que signifie OPCVM ?")
print(response)


llama_model_load_from_file_impl: using device Metal (Apple M1) - 17592186039798 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ../data/models/mistral-7b-instruct-v0.2.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:            

 OPCVM est une abréviation pour Open Payment Card Verification Method. C'est un méthode standardisée utilisée par les banques et les institutions financières pour vérifier les informations fournies sur une carte de paiement, avant d'autoriser le paiement.


In [82]:
print(response)


 OPCVM est une abréviation pour Open Payment Card Verification Method. C'est un méthode standardisée utilisée par les banques et les institutions financières pour vérifier les informations fournies sur une carte de paiement, avant d'autoriser le paiement.


In [83]:
import time

start = time.time()
output = llm("Qui est Napoléon Bonaparte ?")
end = time.time()

print(f"Réponse ({end - start:.2f} sec):\n{output}")


/var/folders/21/zh5wgvsx3cl1p9q6b33y__km0000gn/T/ipykernel_70132/891136700.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm("Qui est Napoléon Bonaparte ?")
Llama.generate: 1 prefix-match hit, remaining 11 prompt tokens to eval
llama_perf_context_print:        load time =  129354.42 ms
llama_perf_context_print: prompt eval time =    3466.67 ms /    11 tokens (  315.15 ms per token,     3.17 tokens per second)
llama_perf_context_print:        eval time =   22175.53 ms /   255 runs   (   86.96 ms per token,    11.50 tokens per second)
llama_perf_context_print:       total time =   25734.48 ms /   266 tokens


Réponse (25.75 sec):


Napoléon Bonaparte (15 août 1769 – 5 mai 1821) was a French military and political leader during the late 18th and early 19th centuries. He is best known for his role as the Emperor of France, from 1804 until 1814, and then again briefly in 1815, before his final defeat at Waterloo.

Napoléon was born on Corsica, a French possession in the Mediterranean Sea. He rose to prominence during the French Revolution, when he served as an officer in the French army.

In the late 1790s and early 1800s, Napoléon gained fame for his military campaigns, which saw him conquer much of Europe. In 1804, Napoléon was crowned Emperor of France, marking the beginning of the Napoleonic Empire.

During the Napoleonic Wars, which lasted from 1803 until 1815, Napoléon's armies fought against those of various European powers, including Austria, Prussia, Russia, Spain


In [85]:
from transformers import AutoTokenizer

# Authentification via token HF
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    use_auth_token=huggin_face_key
)

# Sauvegarde locale
tokenizer.save_pretrained("../data/models/mistral-tokenize")

('../data/models/mistral-tokenize/tokenizer_config.json',
 '../data/models/mistral-tokenize/special_tokens_map.json',
 '../data/models/mistral-tokenize/chat_template.jinja',
 '../data/models/mistral-tokenize/tokenizer.json')

In [5]:
import gradio as gr
import chromadb
import os
from transformers import AutoTokenizer, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.llms import LlamaCpp
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()
CHROMA_DB_PORT = os.getenv("CHROMA_DB_PORT")
CHROMA_DB_HOST = os.getenv("CHROMA_DB_HOST")
CHROMA_DB_COLLECTION = "financial_reports"


embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

client = chromadb.HttpClient(host=CHROMA_DB_HOST, port=CHROMA_DB_PORT)
vectorstore = Chroma(client=client, collection_name="financial_reports", embedding_function=embedding_model)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Contexte : {context}\n\nQuestion : {question}\nRéponse :"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [6]:
docs = retriever.get_relevant_documents("Que signifie OPCVM ?")

/var/folders/21/zh5wgvsx3cl1p9q6b33y__km0000gn/T/ipykernel_14077/2120962751.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Que signifie OPCVM ?")


In [14]:
docs[1].metadata["filename"]

'Allianz.pdf'